In [ ]:
!pip install keras-swa==0.1.5

     |████████████████████████████████| 81kB 10.8MB/s 
  Created wheel for keras-swa: filename=keras_swa-0.1.5-cp37-none-any.whl size=9714 sha256=d4b60e0b854c16b6ce7874311bd41a749d5ff214e64e61b1b2b7c9e55c2103fa
  Stored in directory: /root/.cache/pip/wheels/30/de/a7/f2a71861bb678729a45e8995dc2496118e875e64b89379c530
Successfully built keras-swa


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon May 17 16:38:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from swa.tfkeras import SWA
import tensorflow.keras.backend as K

In [ ]:
from keras.datasets import cifar10

In [ ]:
(trainX, trainy), (testX, testy) = cifar10.load_data()
trainX = trainX/255
testX = testX/255

170500096/170498071 [==============================] - 2s 0us/step


In [ ]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K
from keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

In [ ]:
lb = LabelBinarizer()                                  #Binarize labels in a one-vs-all fashion.
trainy = lb.fit_transform(trainy)                     #Fit label binarizer and transform multi-class labels to binary labels
testy = lb.fit_transform(testy)

In [ ]:
def label_dict(img, labels):  # creating dictionary of labels
  lab_lis = {}
  for i in range(10):
    lab_lis[i] = []          #declaring key values as labels  in dictionary
  data = list(zip(img, labels))  # list of tuples of each image and label pair
  for im, lab in data:
    idx = np.argmax(lab)
    lab_lis[idx].append((im, lab))
  return lab_lis

In [ ]:
t = label_dict(trainX, trainy)

In [ ]:
unlab_data = []
unlab_img_data = []
unlab_img_lab = []
for i in range(len(t)):
  unlab_data.extend(t[i][4000:]) # extracting 1000 imgaes from each class in the training set to generate the pool of so called unlabeled set
for x,y in unlab_data:
  unlab_img_data.append(x)
  unlab_img_lab.append(y)

In [ ]:
client_names = ["client_"+str(i) for i in range(10)]  # makiing non_iid datas set for each client
client_data = {}
for client in client_names:
  client_data[client] = []
i = 0
for client in client_names:
  client_data[client].extend(t[i%10][:1960])
  client_data[client].extend(t[(i+1)%10][1960:3920])
  i += 1
for i in range(10):
  k = 0
  for j in range(10):
    if j != (i)%10 and j != (i+1)%10:
      client_data["client_"+str(j)].extend(t[i][3920 + 10*k: 3920 + 10*(k+1)])
      k += 1

In [ ]:
def batch(zip_data, batch_size = 40):
  data, label = zip(*zip_data)  # Unzipping the img and label pair 
  dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label))) # create a dataset object to make use of other transformation attributes
  return dataset.shuffle(len(label)).batch(batch_size)# shuffles data and creates batches of size 40

In [ ]:
client_batched = {}
for (client_name, data) in client_data.items():
  client_batched[client_name] = batch(data)

test_batched = tf.data.Dataset.from_tensor_slices((testX, testy)).batch(len(testy))
unlab_dataset = tf.data.Dataset.from_tensor_slices((unlab_img_data)).batch(len(unlab_img_data))

In [ ]:

print(list(client_batched.keys()))

['client_0', 'client_1', 'client_2', 'client_3', 'client_4', 'client_5', 'client_6', 'client_7', 'client_8', 'client_9']


In [ ]:
class MLP:
    @staticmethod
    def build():
        tf_input = Input(shape=(32, 32,  3))
        base_model = ResNet50(input_tensor=tf_input, include_top=False)
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(1024, activation='relu')(x)
        predictions = Dense(10, activation='softmax')(x)
        model = Model(inputs=base_model.input, outputs=predictions)
        return model

In [ ]:
model = MLP()
model = model.build()

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
t = model.get_weights()  # (3,3,3,32) => 3 RGB channels, and 32 filters of size (3,3)  => 3*3*3*32 + 32( bias) = 896 (no of param of 1st layer)
                         # (32,) refers to the bias weights 
                         #(3,3,32,32) => 32 filters ouputs of first layer, 32 filters  of size (3,3)of current layer => 32*3*3*32 +32 = 9248
for v in t:                
  print(len(v))
  

7
64
64
64
64
64
1
64
64
64
64
64
3
64
64
64
64
64
1
256
1
256
256
256
256
256
256
256
256
256
1
64
64
64
64
64
3
64
64
64
64
64
1
256
256
256
256
256
1
64
64
64
64
64
3
64
64
64
64
64
1
256
256
256
256
256
1
128
128
128
128
128
3
128
128
128
128
128
1
512
1
512
512
512
512
512
512
512
512
512
1
128
128
128
128
128
3
128
128
128
128
128
1
512
512
512
512
512
1
128
128
128
128
128
3
128
128
128
128
128
1
512
512
512
512
512
1
128
128
128
128
128
3
128
128
128
128
128
1
512
512
512
512
512
1
256
256
256
256
256
3
256
256
256
256
256
1
1024
1
1024
1024
1024
1024
1024
1024
1024
1024
1024
1
256
256
256
256
256
3
256
256
256
256
256
1
1024
1024
1024
1024
1024
1
256
256
256
256
256
3
256
256
256
256
256
1
1024
1024
1024
1024
1024
1
256
256
256
256
256
3
256
256
256
256
256
1
1024
1024
1024
1024
1024
1
256
256
256
256
256
3
256
256
256
256
256
1
1024
1024
1024
1024
1024
1
256
256
256
256
256
3
256
256
256
256
256
1
1024
1024
1024
1024
1024
1
512
512
512
512
512
3
512
512
512
512
512
1
2048
1
2

In [ ]:
model.layers[0].get_config()


{'batch_input_shape': (None, 32, 32, 3),
 'dtype': 'float32',
 'name': 'input_1',
 'ragged': False,
 'sparse': False}

In [ ]:
comms_round = 40
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(lr=0.01, 
                momentum=0.9
               )   
optimizer2 = SGD(lr=0.001,
                 momentum=0.9
               )   

In [ ]:
# model = MLP()
# model = model.build()
# model.compile(loss=loss, 
#                       optimizer=optimizer2, 
#                       metrics=metrics)
# print((model.optimizer.lr))
# model2 = MLP()
# model2 = model2.build()
# model2.compile(loss=loss, 
#                       optimizer=optimizer, 
#                       metrics=metrics)
# print((model2.optimizer.lr))

In [ ]:
weight_scale = []
for (name, data) in client_data.items():
  weight_scale.append(len(data)/40000)
weight_scale = np.array(weight_scale).reshape((len(weight_scale),1))

In [ ]:
# def scale_model_weights(weight, scalar):
#     weight_final = []
#     steps = len(weight)
#     for i in range(steps):
#         weight_final.append(scalar * weight[i])
#     return weight_final

def sum_scaled_weights(scaled_weight_list, M, unlab_data):
    #print(type(scaled_weight_list[0][0]))
    avg_grad_mean = list()
    avg_grad_var = list()
    for grad_list_tuple in zip(*scaled_weight_list):
        #print(np.shape(grad_list_tuple))
        if len(np.shape(grad_list_tuple)) == 5:
          w = weight_scale.reshape((len(weight_scale), 1, 1, 1, 1))
        elif len(np.shape(grad_list_tuple)) == 4:
          w = weight_scale.reshape((len(weight_scale), 1, 1, 1))
        elif len(np.shape(grad_list_tuple)) == 3:
          w = weight_scale.reshape((len(weight_scale), 1, 1))
        else:
          w = weight_scale
        layer_mean = np.sum(grad_list_tuple*w, axis=0)
        #print(np.shape(layer_mean))
        layer_variance = np.sum(np.power(grad_list_tuple - layer_mean, 2)*w, axis = 0) 
        avg_grad_mean.append(layer_mean)
        avg_grad_var.append(layer_variance)
    return avg_grad_mean


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [ ]:
mlp_global = MLP()
global_model = mlp_global.build()

for comm_round in range(comms_round):
  
    global_weights = global_model.get_weights()

    scaled_local_weight_list = list()
    client_names = list(client_batched.keys())
    random.shuffle(client_names)
    
    for client in client_names:
        def scheduler(epoch, lr):
            if epoch  == 8:
              lr = lr*0.1
            if epoch == 15:
              lr = lr*0.1  
            return lr
        mlp_local = MLP()
        local_model = mlp_local.build()
        local_model.compile(loss=loss, 
                      optimizer=SGD(lr=0.01,momentum=0.9), 
                      metrics=metrics)
        local_model.set_weights(global_weights)
        callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
        local_model.fit(client_batched[client], epochs=5, verbose=0, callbacks=[callback])
        scaled_local_weight_list.append(local_model.get_weights())
        K.clear_session()
    avg = sum_scaled_weights(scaled_local_weight_list, 10, unlab_dataset)
    mlp_global = MLP()
    global_model = mlp_global.build()
    global_model.compile(loss=loss, 
                      optimizer=SGD(lr=0.001,momentum=0.9), 
                      metrics=metrics)
    global_model.set_weights(avg)

    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

comm_round: 0 | global_acc: 10.000% | global_loss: 2.3026137351989746
comm_round: 1 | global_acc: 9.990% | global_loss: 2.299663782119751
comm_round: 2 | global_acc: 19.830% | global_loss: 2.281569004058838
comm_round: 3 | global_acc: 34.970% | global_loss: 2.2454729080200195
comm_round: 4 | global_acc: 42.280% | global_loss: 2.2084739208221436
comm_round: 5 | global_acc: 45.920% | global_loss: 2.1750223636627197
comm_round: 6 | global_acc: 52.780% | global_loss: 2.1292898654937744
comm_round: 7 | global_acc: 53.880% | global_loss: 2.108097791671753
comm_round: 8 | global_acc: 55.870% | global_loss: 2.093919277191162
comm_round: 9 | global_acc: 57.450% | global_loss: 2.0696821212768555
comm_round: 10 | global_acc: 58.470% | global_loss: 2.0406908988952637
comm_round: 11 | global_acc: 58.630% | global_loss: 2.036484718322754
comm_round: 12 | global_acc: 59.520% | global_loss: 2.0228590965270996
comm_round: 13 | global_acc: 61.390% | global_loss: 1.9987132549285889
comm_round: 14 | globa